### Modelo Binomial para valuación de opciones Europeas
Enrique Recio

In [1]:
import math
import pandas as pd
import numpy as np
import yfinance as yf

In [2]:
def price_download(stock, a, b, c):
    """
    Descarga precios y crea columna de log_returns
    """
    data = yf.download(stock, start=a, end=b)
    cp = pd.DataFrame(data[c])
    cp['log_ret'] = np.log(cp/cp.shift(1))
    return cp

In [3]:
def binomial(S, K, T, r, vol, N, l):
    """
    Modelo Binomial para valuación de opciones Europeas Call y Put
    S - precio del subyacente
    K - precio del strike
    T - plazo
    r - tasa libre de riesgo
    vol - volatilidad 
    N - numero de pasos en el modelo
    l - equivalente deseado del plazo 
    """
    
    dt = T/360/l
    u =  math.exp(vol * math.sqrt(dt))
    d = 1/u
    p = (math.exp(r * dt) - d)/(u - d)
    C = {}
    
    for m in range(0, N+1):
            C[(N, m)] = max(S * (u ** (2*m - N)) - K, 0)

            
    for k in range(N-1, -1, -1):
        for m in range(0, k+1):
            C[(k, m)] = math.exp(-r * dt) * (p * C[(k+1, m+1)] + (1-p) * C[(k+1, m)])

            
    call = C[(0,0)]
    put = call - S + (K/(math.exp(r*dt) ** N))
     
    return call, put

## Ejercicio 1
Emisora: TSLA
<br>
Plazo: 360
<br>
Analizaremos el comportamiento del precio de la acción durante 5 trimestres (saltos trimestrales)

In [12]:
ticker = 'TSLA'
inicio = '2019-10-07'
final = '2020-10-06'
precio = 'Adj Close'
df = price_download(ticker, inicio, final, precio)

[*********************100%***********************]  1 of 1 completed


In [5]:
S = df[precio].iloc[-1]
K = 415.09
T = 360
r = 0.12/100
vol = df['log_ret'].std()*np.sqrt(T)
N = 5
l = 4

In [6]:
prices = np.round(binomial(S, K, T, r, vol, N, l), decimals = 2)
print(f"{ticker}\nCall: {prices[0]}\nPut: {prices[1]}")

TSLA
Call: 200.68
Put: 189.47


## Ejercicio 2
Emisora: AMZN
<br>
Plazo: 180
<br>
Analizaremos el comportamiento del precio de la acción durante 7 meses (saltos mes a mes)

In [7]:
ticker = 'AMZN'
inicio = '2020-04-06'
final = '2020-10-03'
precio = 'Adj Close'
df = price_download(ticker, inicio, final, precio)

[*********************100%***********************]  1 of 1 completed


In [8]:
S = df[precio].iloc[-1]
K = 3125
T = 180
r = 0.1/100
vol = df['log_ret'].std()*np.sqrt(T)
N = 7
l = 6

In [9]:
prices = binomial(S, K, T, r, vol, N, l)
print(f"{ticker}\nCall: {prices[0]}\nPut: {prices[1]}")

AMZN
Call: 308.9945973970508
Put: 307.17221231104213
